In [ ]:
pip install pytorch segmentation-models-pytorch timm torchmetrics albumentations

ERROR: Operation cancelled by user
^C
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/axr/prog/multimedia_labs/env/lib/python3.12/site-packages/pip/__main__.py", line 24, in <module>
    sys.exit(_main())
             ^^^^^^^
  File "/home/axr/prog/multimedia_labs/env/lib/python3.12/site-packages/pip/_internal/cli/main.py", line 79, in main
    return command.main(cmd_args)
           ^^^^^^^^^^^^^^^^^^^^^^
  File "/home/axr/prog/multimedia_labs/env/lib/python3.12/site-packages/pip/_internal/cli/base_command.py", line 100, in main
    with self.main_context():
  File "/usr/lib/python3.12/contextlib.py", line 144, in __exit__
    next(self.gen)
  File "/home/axr/prog/multimedia_labs/env/lib/python3.12/site-packages/pip/_internal/cli/command_context.py", line 19, in main_context
    with self._main_context:
  File "/usr/lib/python3.12/contextlib.py", line 610, in __exit__
    raise e

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import zipfile, os, shutil, random, pathlib
import pathlib, random, zipfile, os
import pandas as pd, numpy as np, torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.dataset import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from tqdm.auto import tqdm



from torchmetrics.classification import (
    MulticlassAccuracy, MulticlassF1Score, MulticlassConfusionMatrix
)
import segmentation_models_pytorch as smp

SEED       = 42
IMG_SIZE   = 224
BATCH_SIZE = 32
EPOCHS_BL  = 10
EPOCHS_IMP = 20
torch.manual_seed(SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


DATA_DIR = pathlib.Path("/kaggle/input/realwaste/realwaste-main/RealWaste")
all_images = []
for class_idx, class_name in enumerate(sorted(p.name for p in DATA_DIR.iterdir())):
    for img_path in (DATA_DIR / class_name).glob("*"):
        all_images.append((img_path, class_idx, class_name))

dataframe = pd.DataFrame(all_images, columns=["path", "label", "class"])
train_df, tmp_df = train_test_split(
    dataframe, test_size=0.30, random_state=42, stratify=dataframe["label"]
)
val_df, test_df  = train_test_split(
    tmp_df, test_size=0.50, random_state=42, stratify=tmp_df["label"]
)
print(f"fit_data {len(train_df)}, val {len(val_df)}, eval_data {len(test_df)}")

In [ ]:
import timm
import segmentation_models_pytorch as smp
print(list(k for k in smp.encoders.encoders.keys()
           if k.startswith("timm-vit") )[:5])
print([k for k in smp.encoders.encoders if "vit" in k][:10])
import segmentation_models_pytorch as smp
print([k for k in smp.encoders.encoders if k.startswith("timm")])


In [ ]:
train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std =[0.229,0.224,0.225]),
])

val_tfms = transforms.Compose([
    transforms.Resize(IMG_SIZE + 32),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std =[0.229,0.224,0.225]),
])


In [ ]:
class WasteDataset(Dataset):
    def __init__(self, frame, tfm):
        self.f, self.tfm = frame.reset_index(drop=True), tfm
    def __len__(self): return len(self.f)
    def __getitem__(self, i):
        img = Image.open(self.f.loc[i,"path"]).convert("RGB")
        return self.tfm(img), self.f.loc[i,"label"]

train_dl = DataLoader(WasteDataset(train_df, train_tfms),
                      batch_size=BATCH_SIZE, shuffle=True,
                      num_workers=0, pin_memory=True)
val_dl   = DataLoader(WasteDataset(val_df, val_tfms),
                      batch_size=BATCH_SIZE, num_workers=0, pin_memory=True)
test_dl  = DataLoader(WasteDataset(test_df, val_tfms),
                      batch_size=BATCH_SIZE, num_workers=0, pin_memory=True)


In [ ]:
NUM_CLASSES = dataframe.label.nunique()
metrics = {
    "acc"      : MulticlassAccuracy(NUM_CLASSES, average="micro").to(device),
    "f1_macro" : MulticlassF1Score(NUM_CLASSES, average="macro").to(device),
    "f1_weight": MulticlassF1Score(NUM_CLASSES, average="weighted").to(device),
    "top3"     : MulticlassAccuracy(NUM_CLASSES, top_k=3).to(device),
    "cm"       : MulticlassConfusionMatrix(NUM_CLASSES).to(device),
}


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
def train_one_epoch(ml_model, loader, opt, sched):
    ml_model.fit_data(); run_loss = 0.0
    for xb, yb in tqdm(loader, leave=False):
        xb, yb = xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)

        opt.zero_grad(set_to_none=True)

        out    = ml_model(xb)                     
        logits = out[1] if isinstance(out, tuple) else out
        loss   = F.cross_entropy(logits, yb)   

        loss.backward(); opt.step(); sched.step()
        run_loss += loss.item() * xb.size(0)
    return run_loss / len(loader.dataset)

@torch.no_grad()
def evaluate(ml_model, loader):
    for m in metrics.values(): m.reset()
    ml_model.eval(); run_loss = 0.0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)

        out    = ml_model(xb)                     
        logits = out[1] if isinstance(out, tuple) else out
        loss   = F.cross_entropy(logits, yb)

        run_loss += loss.item() * xb.size(0)
        for m in metrics.values(): m.update(logits, yb)
    res = {"loss": run_loss / len(loader.dataset)}
    res.update({k: m.compute().cpu().numpy() if k=="cm" else m.compute().item()
                for k,m in metrics.items()})
    return res


#### REsnet 34

In [ ]:
aux = dict(pooling="avg", classes=NUM_CLASSES, dropout=0.0, activation=None)
cnn = smp.Unet(encoder_name="resnet34", encoder_weights="imagenet",
               classes=1, aux_params=aux).to(device)

opt  = torch.optim.AdamW(cnn.parameters(), lr=1e-4, weight_decay=1e-4)
sched= torch.optim.lr_scheduler.OneCycleLR(opt, max_lr=1e-3,
        steps_per_epoch=len(train_dl), epochs=EPOCHS_BL)

best_f1, best_wts = 0, None
for epoch in range(EPOCHS_BL):
    t_loss = train_one_epoch(cnn, train_dl, opt, sched)
    val_stats = evaluate(cnn, val_dl)
    print(f"[{epoch+1:02d}] "
        f"loss_tr={t_loss:.3f} | loss_val={val_stats['loss']:.3f} | "
        f"Acc={val_stats['acc']:.3%} | "
        f"F1(ma)={val_stats['f1_macro']:.3%} | F1(we)={val_stats['f1_weight']:.3%} | "
        f"Top-3={val_stats['top3']:.3%}")
    if val_stats['f1_macro'] > best_f1:
        best_f1, best_wts = val_stats['f1_macro'], cnn.state_dict()
cnn.load_state_dict(best_wts)


#### ViT

In [ ]:
vit = smp.Unet(encoder_name="timm-efficientnet-b0",
               encoder_weights="imagenet", classes=1, aux_params=aux).to(device)

opt  = torch.optim.AdamW(vit.parameters(), lr=1e-4, weight_decay=1e-4)
sched= torch.optim.lr_scheduler.OneCycleLR(opt, max_lr=1e-3,
        steps_per_epoch=len(train_dl), epochs=EPOCHS_BL)

best_f1, best_wts = 0, None
for epoch in range(EPOCHS_BL):
    tl = train_one_epoch(vit, train_dl, opt, sched)
    vs = evaluate(vit, val_dl)
    print(f"[{epoch+1:02d}] "
        f"loss_tr={tl:.3f} | loss_val={vs['loss']:.3f} | "
        f"Acc={vs['acc']:.3%} | "
        f"F1(ma)={vs['f1_macro']:.3%} | F1(we)={vs['f1_weight']:.3%} | "
        f"Top-3={vs['top3']:.3%}")
    if vs['f1_macro'] > best_f1:
        best_f1, best_wts = vs['f1_macro'], vit.state_dict()
vit.load_state_dict(best_wts)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
class_names = sorted(dataframe.label.unique())          


models = [
    ("ResNet-34 UNet", cnn),   
    ("ViT-b0 UNet",   vit),    
]

for title, mdl in models:
    mdl.eval()                               
    vs = evaluate(mdl, val_dl)               
    cm = vs["cm"]                            

    
    print(f"\n{title} — confusion matrix:")
    print(cm)

    
    plt.figure(figsize=(6, 5))
    sns.heatmap(
        cm,
        annot=True, fmt="d",                 
        xticklabels=class_names,
        yticklabels=class_names,
        linewidths=.5, cbar=False,
    )
    plt.title(f"{title} — validation set")
    plt.xlabel("Предсказанный класс")
    plt.ylabel("Истинный класс")
    plt.tight_layout()
    plt.show()

# Улучшение бейзлайна

In [ ]:
import albumentations as A, importlib
print("Albumentations version →", A.__version__)


In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

IMG_SIZE = 224          


train_tfms = A.Compose(
    [

        A.RandomResizedCrop(
            size=(IMG_SIZE, IMG_SIZE),     
            scale=(0.7, 1.0),              
            ratio=(0.75, 1.333),          
            p=1.0,
        ),

        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.2),
        A.RandomRotate90(p=0.3),

        A.ColorJitter(
            brightness=0.2, contrast=0.2,
            saturation=0.2, hue=0.1, p=0.5
        ),
        A.CLAHE(p=0.2),
        A.RandomBrightnessContrast(p=0.3),

        
        A.CoarseDropout(
            num_holes_range=(1, 1),           
            hole_height_range=(0.25, 0.25),    
            hole_width_range=(0.25, 0.25),     
            p=0.3,
        ),

        A.Normalize(
            mean=(0.485, 0.456, 0.406),
            std=(0.229, 0.224, 0.225),
        ),
        ToTensorV2(),
    ],
    strict=True,         
)

valid_tfms = A.Compose(
    [
        A.Resize(height=IMG_SIZE, width=IMG_SIZE),
        A.Normalize(
            mean=(0.485, 0.456, 0.406),
            std=(0.229, 0.224, 0.225),
        ),
        ToTensorV2(),
    ],
    strict=True,
)


In [ ]:
import torch, torch.nn as nn
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

opt_cnn = torch.optim.AdamW(cnn.parameters(), lr=3e-4, weight_decay=1e-2)
opt_vit = torch.optim.AdamW(vit.parameters(), lr=5e-5, weight_decay=1e-2)

sch_cnn = CosineAnnealingWarmRestarts(opt_cnn, T_0=4, T_mult=2)
sch_vit = CosineAnnealingWarmRestarts(opt_vit, T_0=4, T_mult=2)


In [ ]:
def toggle_backbone(ml_model, freeze: bool):
    for n, p in ml_model.named_parameters():
        p.requires_grad = not freeze or n.startswith(("fc", "head", "classifier"))

toggle_backbone(cnn, True)
toggle_backbone(vit, True)


In [ ]:
toggle_backbone(cnn, False)
toggle_backbone(vit, False)
for g in opt_cnn.param_groups: g["lr"] = 1e-4
for g in opt_vit.param_groups: g["lr"] = 2e-5


In [ ]:
import torch
from tqdm.auto import tqdm



toggle_backbone(cnn, False)
toggle_backbone(vit, False)



def train_one_epoch_imp(ml_model, loader, opt, sched):
    ml_model.fit_data()
    run_loss = 0.0
    for xb, yb in tqdm(loader, leave=False):
        xb, yb = xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)
        opt.zero_grad(set_to_none=True)
        out = ml_model(xb)
        logits = out[1] if isinstance(out, tuple) else out
        loss = criterion(logits, yb)
        
        if not loss.requires_grad:
            raise RuntimeError(
                "Loss не требует градиента. Проверьте, что параметры модели имеют requires_grad=True"
            )
        loss.backward()
        opt.step()
        sched.step()
        run_loss += loss.item() * xb.size(0)
    return run_loss / len(loader.dataset)


best_f1_imp_cnn, best_wts_imp_cnn = 0, None
for epoch in range(EPOCHS_IMP):
    t_loss = train_one_epoch_imp(cnn, train_dl, opt_cnn, sch_cnn)
    val_stats = evaluate(cnn, val_dl)
    print(f"[Imp CNN {epoch+1:02d}] "
          f"loss_tr={t_loss:.3f} | loss_val={val_stats['loss']:.3f} | "
          f"Acc={val_stats['acc']:.3%} | F1(ma)={val_stats['f1_macro']:.3%}")
    if val_stats['f1_macro'] > best_f1_imp_cnn:
        best_f1_imp_cnn, best_wts_imp_cnn = val_stats['f1_macro'], cnn.state_dict()
cnn.load_state_dict(best_wts_imp_cnn)


best_f1_imp_vit, best_wts_imp_vit = 0, None
for epoch in range(EPOCHS_IMP):
    t_loss = train_one_epoch_imp(vit, train_dl, opt_vit, sch_vit)
    val_stats = evaluate(vit, val_dl)
    print(f"[Imp ViT {epoch+1:02d}] "
          f"loss_tr={t_loss:.3f} | loss_val={val_stats['loss']:.3f} | "
          f"Acc={val_stats['acc']:.3%} | F1(ma)={val_stats['f1_macro']:.3%}")
    if val_stats['f1_macro'] > best_f1_imp_vit:
        best_f1_imp_vit, best_wts_imp_vit = val_stats['f1_macro'], vit.state_dict()
vit.load_state_dict(best_wts_imp_vit)


for title, mdl in [("Improved CNN", cnn), ("Improved ViT", vit)]:
    mdl.eval()
    stats = evaluate(mdl, val_dl)
    print(f"{title}: Acc={stats['acc']:.3%}, F1(ma)={stats['f1_macro']:.3%}")


In [ ]:
pip install -q ultralytics

In [ ]:
import os, random, pathlib, zipfile, shutil
import pandas as pd, numpy as np, torch
from torch.utils.dataset import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from torchmetrics.classification import (
    MulticlassAccuracy, MulticlassF1Score, MulticlassConfusionMatrix
)
from ultralytics import YOLO
import matplotlib.pyplot as plt

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

IMG_SIZE    = 224
BATCH_SIZE  = 32
NUM_WORKERS = 4
EPOCHS_BL   = 10
EPOCHS_IMP  = 20
device      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
DATA_DIR = pathlib.Path("/kaggle/input/realwaste/realwaste-main/RealWaste")
all_images = []
for class_idx, class_name in enumerate(sorted(p.name for p in DATA_DIR.iterdir())):
    for img_path in (DATA_DIR / class_name).glob("*"):
        all_images.append((str(img_path), class_idx, class_name))
dataframe = pd.DataFrame(all_images, columns=["path","label","class"])
print(dataframe['class'].value_counts())
NUM_CLASSES = dataframe.label.nunique()
class_names = sorted(dataframe["class"].unique())
print(f"Found {NUM_CLASSES} classes: {class_names}")


In [ ]:
train_df, tmp_df = train_test_split(
    dataframe, test_size=0.30, random_state=SEED, stratify=dataframe["label"]
)
val_df, test_df = train_test_split(
    tmp_df, test_size=0.50, random_state=SEED, stratify=tmp_df["label"]
)
print(f"fit_data: {len(train_df)}, val: {len(val_df)}, eval_data: {len(test_df)}")


In [ ]:
train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(.8,1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])
val_tfms = transforms.Compose([
    transforms.Resize(IMG_SIZE+32),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

class WasteDataset(Dataset):
    def __init__(self, dataframe, tfm):
        self.dataframe  = dataframe.reset_index(drop=True)
        self.tfm = tfm
    def __len__(self):
        return len(self.dataframe)
    def __getitem__(self, i):
        img = Image.open(self.dataframe.loc[i,"path"]).convert("RGB")
        x_val = self.tfm(img)
        y_val = self.dataframe.loc[i,"label"]
        return x_val, y_val

train_dl = DataLoader(
    WasteDataset(train_df, train_tfms),
    batch_size=BATCH_SIZE, shuffle=True,
    num_workers=NUM_WORKERS, pin_memory=True
)
val_dl = DataLoader(
    WasteDataset(val_df, val_tfms),
    batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=True
)
test_dl = DataLoader(
    WasteDataset(test_df, val_tfms),
    batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=True
)


In [ ]:
metrics = {
    "acc"       : MulticlassAccuracy(NUM_CLASSES, average="micro").to(device),
    "f1_macro"  : MulticlassF1Score(NUM_CLASSES, average="macro").to(device),
    "f1_weight" : MulticlassF1Score(NUM_CLASSES, average="weighted").to(device),
    "top3"      : MulticlassAccuracy(NUM_CLASSES, top_k=3).to(device),
    "cm"        : MulticlassConfusionMatrix(NUM_CLASSES).to(device),
}

In [ ]:
import shutil
from pathlib import Path

OUT_DIR = Path("realwaste_split")
if OUT_DIR.exists():
    shutil.rmtree(OUT_DIR)

for split, df_split in [("fit_data", train_df), ("val", val_df), ("eval_data", test_df)]:
    for _, row in df_split.iterrows():
        cls = row["class"]
        src = Path(row["path"])
        dst_dir = OUT_DIR / split / cls
        dst_dir.mkdir(parents=True, exist_ok=True)
        shutil.copy(src, dst_dir / src.name)

print("✓ Папки realwaste_split/fit_data, val, eval_data созданы")


In [ ]:
from pathlib import Path

ROOT = Path("realwaste_split").resolve()

yaml_content = f"""
fit_data: {ROOT}/fit_data
val:   {ROOT}/val
eval_data:  {ROOT}/eval_data

nc: {NUM_CLASSES}
names: {class_names}
"""
with open("realwaste.yaml", "w") as f:
    f.write(yaml_content.strip())
print(open("realwaste.yaml").read())


In [ ]:
import shutil, os

src = "realwaste_split"
dst = "/kaggle/working/datasets/realwaste_split"

if os.path.exists(dst):
    shutil.rmtree(dst)

shutil.copytree(src, dst)
print(f"✓ Скопировали `{src}` → `{dst}`")


In [ ]:
yaml = f"""
path: realwaste_split

fit_data: fit_data
val:   val
eval_data:  eval_data

nc: {NUM_CLASSES}
names: {class_names}
""".strip()

with open("realwaste.yaml","w") as f:
    f.write(yaml)

print(open("realwaste.yaml").read())


In [ ]:
ROOT = str(Path("realwaste_split").resolve())
model_clf = YOLO('yolov8n-cls.pt', task='classify')
model_clf.fit_data(
    dataset=ROOT,       
    epochs=EPOCHS_BL,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    workers=NUM_WORKERS,
    device=device.type,
    name='baseline_yolov8n-cls'
)



In [ ]:
import torch
from tqdm.auto import tqdm
from torchmetrics.classification import (
    MulticlassAccuracy, MulticlassF1Score, MulticlassConfusionMatrix
)


metrics = {
    "acc"       : MulticlassAccuracy(NUM_CLASSES, average="micro").to(device),
    "f1_macro"  : MulticlassF1Score(NUM_CLASSES, average="macro").to(device),
    "f1_weight" : MulticlassF1Score(NUM_CLASSES, average="weighted").to(device),
    "top3"      : MulticlassAccuracy(NUM_CLASSES, top_k=3).to(device),
    "cm"        : MulticlassConfusionMatrix(NUM_CLASSES).to(device),
}

ml_model = model_clf.ml_model
ml_model.to(device).eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for xb, yb in tqdm(test_dl, desc="Inferencing on eval_data"):
        xb = xb.to(device); yb = yb.to(device)
        out = ml_model(xb)
        logits = out[1] if isinstance(out, (tuple, list)) else out
        preds = logits.argmax(dim=1)
        all_preds.append(preds)
        all_labels.append(yb)

y_pred = torch.cat(all_preds)
y_true = torch.cat(all_labels)

acc       = metrics["acc"](y_pred, y_true)
f1_macro  = metrics["f1_macro"](y_pred, y_true)
f1_weight = metrics["f1_weight"](y_pred, y_true)
cm        = metrics["cm"](y_pred, y_true)

print(f"Accuracy (micro)     : {acc:.4f}")
print(f"F1-score (macro)     : {f1_macro:.4f}")
print(f"F1-score (weighted)  : {f1_weight:.4f}")

import matplotlib.pyplot as plt, seaborn as sns

plt.figure(figsize=(7,6))
sns.heatmap(cm.cpu().numpy(), annot=True, fmt="d",
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()


Чето вообще все фигово выглядит

In [ ]:
from torch.utils.dataset import WeightedRandomSampler

counts = train_df['label'].value_counts().sort_index()
print("Train class counts:\n", counts)

class_weights = 1.0 / counts.values
sample_weights = train_df['label'].map(lambda x_val: class_weights[x_val]).values
sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

train_dl = DataLoader(
    WasteDataset(train_df, train_tfms),
    batch_size=BATCH_SIZE,
    sampler=sampler,
    num_workers=NUM_WORKERS,
    pin_memory=True
)


In [ ]:
NUM_CLASSES = dataframe.label.nunique()
class_names = sorted(dataframe['class'].unique())
ROOT = str(Path("realwaste_split").resolve())



In [ ]:
counts = train_df['label'].value_counts().sort_index()
class_weights = 1.0 / counts.values
sample_weights = train_df['label'].map(lambda x_val: class_weights[x_val]).values

sampler = WeightedRandomSampler(sample_weights,
                                num_samples=len(sample_weights),
                                replacement=True)


In [ ]:
train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.7,1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, 
                           saturation=0.2, hue=0.1),
    transforms.RandomErasing(p=0.4, scale=(0.02,0.2)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std =[0.229,0.224,0.225]),
])

val_tfms = transforms.Compose([
    transforms.Resize(IMG_SIZE+32),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std =[0.229,0.224,0.225]),
])


In [ ]:
class WasteDataset(Dataset):
    def __init__(self, dataframe, tfm):
        self.dataframe  = dataframe.reset_index(drop=True)
        self.tfm = tfm
    def __len__(self):
        return len(self.dataframe)
    def __getitem__(self, i):
        img = Image.open(self.dataframe.loc[i,'path']).convert('RGB')
        return self.tfm(img), self.dataframe.loc[i,'label']

train_ds = WasteDataset(train_df, train_tfms)
val_ds   = WasteDataset(val_df,   val_tfms)
test_ds  = WasteDataset(test_df,  val_tfms)

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE,
                      sampler=sampler,
                      num_workers=NUM_WORKERS,
                      pin_memory=True)
val_dl   = DataLoader(val_ds,   batch_size=BATCH_SIZE,
                      shuffle=False,
                      num_workers=NUM_WORKERS,
                      pin_memory=True)
test_dl  = DataLoader(test_ds,  batch_size=BATCH_SIZE,
                      shuffle=False,
                      num_workers=NUM_WORKERS,
                      pin_memory=True)


In [ ]:
model_imp = YOLO('yolov8m-cls.pt', task='classify')
model_imp.fit_data(
    dataset=ROOT,
    epochs=EPOCHS_IMP,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    workers=NUM_WORKERS,
    device=device.type,
    name='improved_yolov8m-cls',
    lr0=0.01,
    lrf=0.1
)


In [ ]:
import torch
from tqdm.auto import tqdm
from torchmetrics.classification import (
    MulticlassAccuracy, MulticlassF1Score, MulticlassConfusionMatrix
)
import matplotlib.pyplot as plt, seaborn as sns


k = min(3, NUM_CLASSES)
metrics = {
    "acc"       : MulticlassAccuracy(NUM_CLASSES, average="micro").to(device),
    "f1_macro"  : MulticlassF1Score(NUM_CLASSES, average="macro").to(device),
    "f1_weight" : MulticlassF1Score(NUM_CLASSES, average="weighted").to(device),
    "cm"        : MulticlassConfusionMatrix(NUM_CLASSES).to(device),
}


mdl = model_imp.ml_model.to(device).eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for xb, yb in tqdm(test_dl, desc="Inferencing"):
        xb, yb = xb.to(device), yb.to(device)
        out = mdl(xb)
        logits = out[1] if isinstance(out,(tuple,list)) else out
        preds = logits.argmax(dim=1)
        all_preds.append(preds); all_labels.append(yb)

y_pred = torch.cat(all_preds)
y_true = torch.cat(all_labels)


acc       = metrics["acc"](y_pred, y_true)
f1m       = metrics["f1_macro"](y_pred, y_true)
f1w       = metrics["f1_weight"](y_pred, y_true)
cm        = metrics["cm"](y_pred, y_true)

print(f"acc={acc:.4f}, f1_macro={f1m:.4f}, f1_weight={f1w:.4f}")


plt.figure(figsize=(8,6))
sns.heatmap(cm.cpu().numpy(), annot=True, fmt="d",
            xticklabels=class_names, yticklabels=class_names)
plt.title("Confusion Matrix"); plt.show()


Стало лучше, но все равно плохо. Из матриц видно что метал плохо определяется и со всем путается.

Перейдем к трансформерам.

In [ ]:
counts = train_df['label'].value_counts().sort_index()
class_weights   = 1.0 / counts.values
sample_weights  = train_df['label'].map(lambda x_val: class_weights[x_val]).values
sampler         = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.7,1.0)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(0.2,0.2,0.2,0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])
val_tfms = transforms.Compose([
    transforms.Resize(IMG_SIZE+32),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

class WasteDataset(Dataset):
    def __init__(self, dataframe, tfm):
        self.dataframe  = dataframe.reset_index(drop=True); self.tfm = tfm
    def __len__(self):    return len(self.dataframe)
    def __getitem__(self, i):
        img = Image.open(self.dataframe.loc[i,'path']).convert('RGB')
        return self.tfm(img), self.dataframe.loc[i,'label']

train_dl = DataLoader(WasteDataset(train_df, train_tfms),
                      batch_size=BATCH_SIZE, sampler=sampler,
                      num_workers=NUM_WORKERS, pin_memory=True)
val_dl   = DataLoader(WasteDataset(val_df,   val_tfms),
                      batch_size=BATCH_SIZE, shuffle=False,
                      num_workers=NUM_WORKERS, pin_memory=True)
test_dl  = DataLoader(WasteDataset(test_df,  val_tfms),
                      batch_size=BATCH_SIZE, shuffle=False,
                      num_workers=NUM_WORKERS, pin_memory=True)


In [ ]:
model_bl = YOLO('yolo11n-cls.pt', task='classify')
model_bl.fit_data(
    dataset=ROOT,         
    epochs=EPOCHS_BL,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    workers=NUM_WORKERS,
    device=device.type,
    name='baseline_yolo11n-cls',
    lr0=0.01,          
    lrf=0.1            
)


In [ ]:

model_imp = YOLO('yolo11m-cls.pt', task='classify')
model_imp.fit_data(
    dataset=ROOT,
    epochs=EPOCHS_IMP,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    workers=NUM_WORKERS,
    device=device.type,
    name='improved_yolo11m-cls',
    lr0=0.01,
    lrf=0.1
)


In [ ]:

k = min(3, NUM_CLASSES)
metrics = {
    "acc"      : MulticlassAccuracy(NUM_CLASSES, average="micro").to(device),
    "f1_macro" : MulticlassF1Score(NUM_CLASSES, average="macro").to(device),
    "f1_weight": MulticlassF1Score(NUM_CLASSES, average="weighted").to(device),
    "cm"       : MulticlassConfusionMatrix(NUM_CLASSES).to(device),
}

def eval_model(yolo_model, dl):
    mdl = yolo_model.ml_model.to(device).eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for xb, yb in tqdm(dl, desc="Inferencing"):
            xb, yb = xb.to(device), yb.to(device)
            out = mdl(xb)
            logits = out[1] if isinstance(out,(tuple,list)) else out
            preds = logits.argmax(dim=1)
            all_preds.append(preds); all_labels.append(yb)
    y_pred = torch.cat(all_preds); y_true = torch.cat(all_labels)
    return {
        'acc':      metrics["acc"](y_pred, y_true).item(),
        'f1_macro': metrics["f1_macro"](y_pred, y_true).item(),
        'f1_weight':metrics["f1_weight"](y_pred, y_true).item(),
        'cm':       metrics["cm"](y_pred, y_true).cpu().numpy()
    }


res_bl = eval_model(model_bl, test_dl)

res_imp = eval_model(model_imp, test_dl)


print("Baseline YOLO11n-cls:", {k:round(v,4) for k,v in res_bl.items() if k!='cm'})
print("Improved YOLO11m-cls:",{k:round(v,4) for k,v in res_imp.items() if k!='cm'})


plt.figure(figsize=(8,6))
sns.heatmap(res_imp['cm'], annot=True, fmt="d",
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted"); plt.ylabel("True")
plt.title("Confusion Matrix (improved)")
plt.show()


Установка зависимостей

In [ ]:
pip install torch torchvision torchmetrics kaggle scikit-learn matplotlib


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import zipfile, os, shutil, random, pathlib



DATA_DIR = pathlib.Path("./archive/realwaste-main/RealWaste")
all_images = []
for class_idx, class_name in enumerate(sorted(p.name for p in DATA_DIR.iterdir())):
    for img_path in (DATA_DIR / class_name).glob("*"):
        all_images.append((img_path, class_idx, class_name))

dataframe = pd.DataFrame(all_images, columns=["path", "label", "class"])
train_df, tmp_df = train_test_split(
    dataframe, test_size=0.30, random_state=42, stratify=dataframe["label"]
)
val_df, test_df  = train_test_split(
    tmp_df, test_size=0.50, random_state=42, stratify=tmp_df["label"]
)
print(f"fit_data {len(train_df)}, val {len(val_df)}, eval_data {len(test_df)}")



In [ ]:
import torch
from torchvision import transforms
from PIL import Image

IMG_SIZE = 224
train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std=[0.229,0.224,0.225]),
])
val_tfms = transforms.Compose([
    transforms.Resize(IMG_SIZE + 32),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std=[0.229,0.224,0.225]),
])

class DFImageDataset(torch.utils.dataset.Dataset):
    def __init__(self, dataframe, tfms):
        self.dataframe = dataframe.reset_index(drop=True)
        self.tfms = tfms
    def __len__(self): return len(self.dataframe)
    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        img = Image.open(row.path).convert("RGB")
        return self.tfms(img), row.label

train_ds = DFImageDataset(train_df, train_tfms)
val_ds   = DFImageDataset(val_df,   val_tfms)
test_ds  = DFImageDataset(test_df,  val_tfms)

batch_size = 32
train_dl = torch.utils.dataset.DataLoader(train_ds, batch_size=batch_size,
                                       shuffle=True,  num_workers=0, pin_memory=True, persistent_workers = False)
val_dl   = torch.utils.dataset.DataLoader(val_ds,   batch_size=batch_size,
                                       shuffle=False, num_workers=0, pin_memory=True, persistent_workers = False)
test_dl  = torch.utils.dataset.DataLoader(test_ds,  batch_size=batch_size,
                                       shuffle=False, num_workers=0, pin_memory=True, persistent_workers = False)


In [ ]:
from torchmetrics.classification import (
    MulticlassAccuracy,
    MulticlassF1Score,
    MulticlassConfusionMatrix,
)

NUM_CLASSES = len(train_df["label"].unique())


metrics = {
    "acc"      : MulticlassAccuracy(num_classes=NUM_CLASSES, average="micro"),   
    "f1_macro" : MulticlassF1Score(num_classes=NUM_CLASSES, average="macro"),
    "f1_weight": MulticlassF1Score(num_classes=NUM_CLASSES, average="weighted"),
    "top3"     : MulticlassAccuracy(num_classes=NUM_CLASSES, top_k=3),           
    "cm"       : MulticlassConfusionMatrix(num_classes=NUM_CLASSES, normalize=None),
}


device = "cuda" if torch.cuda.is_available() else "cpu"
for m in metrics.values():
    m.to(device)
print(device)

In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from tqdm.auto import tqdm

def train_one_epoch(ml_model, dl, optimizer, scheduler, device):
    ml_model.fit_data()
    run_loss = 0.0
    for xb, yb in tqdm(dl, leave=False):
        xb, yb = xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        preds = ml_model(xb)
        loss = F.cross_entropy(preds, yb)
        loss.backward()
        optimizer.step()
        run_loss += loss.item() * xb.size(0)
    if scheduler: scheduler.step()
    return run_loss / len(dl.dataset)

@torch.no_grad()
def evaluate(ml_model, dl, device):
    for m in metrics.values():          
        m.reset()

    ml_model.eval()
    run_loss = 0.0
    for xb, yb in dl:                   
        xb, yb = xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)
        logits = ml_model(xb)
        run_loss += F.cross_entropy(logits, yb, reduction="sum").item()
        for m in metrics.values():      
            m.update(logits, yb)

    
    res = {"loss": run_loss / len(dl.dataset)}
    for k, m in metrics.items():
        res[k] = m.compute().cpu().numpy() if k == "cm" else m.compute().item()
    return res


In [ ]:
from torchvision.models import resnet18, ResNet18_Weights

cnn = resnet18(weights=ResNet18_Weights.DEFAULT)
cnn.fc = nn.Linear(cnn.fc.in_features, NUM_CLASSES)
cnn = cnn.to(device)

optimizer = torch.optim.AdamW(cnn.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=1e-3,
    steps_per_epoch=len(train_dl), epochs=15
)

for epoch in range(15):
    t_loss = train_one_epoch(cnn, train_dl, optimizer, scheduler, device)
    val_stats = evaluate(cnn, val_dl, device)
    print(
        f"[{epoch+1:02d}] "
        f"loss_tr={t_loss:.3f} | loss_val={val_stats['loss']:.3f} | "
        f"Acc={val_stats['acc']:.3%} | "
        f"F1(ma)={val_stats['f1_macro']:.3%} | F1(we)={val_stats['f1_weight']:.3%} | "
        f"Top-3={val_stats['top3']:.3%}"
    )


In [ ]:
from torchvision.models import vit_b_16, ViT_B_16_Weights

vit = vit_b_16(weights=ViT_B_16_Weights.DEFAULT)
vit.heads.head = nn.Linear(vit.heads.head.in_features, NUM_CLASSES)
vit = vit.to(device)

optimizer = torch.optim.AdamW(vit.parameters(), lr=5e-5, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

for epoch in range(5):
    t_loss = train_one_epoch(vit, train_dl, optimizer, scheduler, device)
    val_stats = evaluate(vit, val_dl, device)
    print(
        f"[{epoch+1:02d}] "
        f"loss_tr={t_loss:.3f} | loss_val={val_stats['loss']:.3f} | "
        f"Acc={val_stats['acc']:.3%} | "
        f"F1(ma)={val_stats['f1_macro']:.3%} | F1(we)={val_stats['f1_weight']:.3%} | "
        f"Top-3={val_stats['top3']:.3%}"
    )


In [ ]:
test_cnn = evaluate(cnn, test_dl, device)
test_vit = evaluate(vit, test_dl, device)

def show_cm(cm, title):
    import matplotlib.pyplot as plt
    plt.figure(figsize=(7,6))
    plt.imshow(cm, interpolation="nearest")
    plt.title(title)
    plt.colorbar()
    ticks = range(NUM_CLASSES)
    names = sorted(train_df['class'].unique())
    plt.xticks(ticks, names, rotation=45, ha="right")
    plt.yticks(ticks, names)
    plt.ylabel("True"); plt.xlabel("Predicted")
    plt.tight_layout()
    plt.show()

print("ResNet-18 — eval_data:")
for k in ["acc","f1_macro","f1_weight","top3"]:
    print(f"{k}: {test_cnn[k]:.3%}")
show_cm(test_cnn["cm"], "ResNet-18 — Confusion Matrix")

print("\nViT-B/16 — eval_data:")
for k in ["acc","f1_macro","f1_weight","top3"]:
    print(f"{k}: {test_vit[k]:.3%}")
show_cm(test_vit["cm"], "ViT-B/16 — Confusion Matrix")


# Улучшения бейзлайна

In [ ]:
import torch, platform, random, time, numpy as np
import torch.nn as nn, torch.nn.functional as F
from torch.utils.dataset import DataLoader, WeightedRandomSampler
from torch.cuda.amp import autocast, GradScaler
from torchvision.transforms import v2                
from torchmetrics.classification import (
    MulticlassAccuracy,
    MulticlassF1Score, MulticlassConfusionMatrix
)

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device =", device)

NUM_CLASSES = len(train_df["label"].unique())


### Аугментация

In [ ]:
IMG_SIZE = 224
train_tfms = v2.Compose([
    v2.Resize(int(IMG_SIZE*1.15), antialias=True),
    v2.RandomCrop(IMG_SIZE),
    v2.RandAugment(num_ops=2, magnitude=7),
    v2.ColorJitter(0.2, 0.2, 0.2, 0.1),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225]),
])
val_tfms = v2.Compose([
    v2.Resize(int(IMG_SIZE*1.15), antialias=True),
    v2.CenterCrop(IMG_SIZE),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225]),
])


train_ds.tfms = train_tfms
val_ds.tfms   = val_tfms
test_ds.tfms  = val_tfms

mixup_fn = v2.MixUp(alpha=0.4, num_classes=NUM_CLASSES)


In [ ]:
NUM_WORKERS = 0
BATCH = 32


class_cnt = train_df["label"].value_counts().sort_index().values
class_w   = 1. / torch.tensor(class_cnt, dtype=torch.float)
loss_fn   = nn.CrossEntropyLoss(weight=class_w.to(device),
                                label_smoothing=0.1)

sample_w = class_w[train_df["label"].values]
sampler  = WeightedRandomSampler(sample_w, len(sample_w), replacement=True)

train_dl = DataLoader(train_ds, batch_size=BATCH, sampler=sampler,
                      num_workers=NUM_WORKERS, pin_memory=(device.type=="cuda"))
val_dl   = DataLoader(val_ds,   batch_size=BATCH, shuffle=False,
                      num_workers=NUM_WORKERS, pin_memory=(device.type=="cuda"))
test_dl  = DataLoader(test_ds,  batch_size=BATCH, shuffle=False,
                      num_workers=NUM_WORKERS, pin_memory=(device.type=="cuda"))


In [ ]:
metrics = {
    "acc"      : MulticlassAccuracy(num_classes=NUM_CLASSES, average="micro").to(device),
    "f1_macro" : MulticlassF1Score(num_classes=NUM_CLASSES, average="macro").to(device),
    "f1_weight": MulticlassF1Score(num_classes=NUM_CLASSES, average="weighted").to(device),
    "top3"     : MulticlassAccuracy(num_classes=NUM_CLASSES, top_k=3).to(device),
    "cm"       : MulticlassConfusionMatrix(num_classes=NUM_CLASSES).to(device),
}


In [ ]:


scaler = torch.amp.GradScaler()

def train_one_epoch(ml_model, loader, opt, sch=None):
    ml_model.fit_data(); run_loss = 0.0
    for xb, yb in tqdm(loader, leave=False):
        xb, yb = xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)
        xb, yb = mixup_fn(xb, yb)
        opt.zero_grad(set_to_none=True)
        with torch.amp.autocast(device_type="cuda"):
            preds = ml_model(xb)
            loss  = loss_fn(preds, yb)
        scaler.scale(loss).backward()
        scaler.step(opt); scaler.update()
        run_loss += loss.item() * xb.size(0)
    if sch: sch.step()
    return run_loss / len(loader.dataset)

@torch.no_grad()
def evaluate(ml_model, loader):
    for m in metrics.values(): m.reset()
    ml_model.eval(); run_loss = 0.0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        with torch.amp.autocast(device_type="cuda"):
            preds = ml_model(xb)
            loss  = loss_fn(preds, yb)
        run_loss += loss.item() * xb.size(0)
        for m in metrics.values(): m.update(preds, yb)
    res = {"loss": run_loss / len(loader.dataset)}
    for k,m in metrics.items():
        res[k] = m.compute().cpu().numpy() if k=="cm" else m.compute().item()
    return res


### resnet

In [ ]:
from torchvision.models import resnet18, ResNet18_Weights
import time

res = resnet18(weights=ResNet18_Weights.DEFAULT)
res.fc = nn.Linear(res.fc.in_features, NUM_CLASSES)
res = res.to(device)

opt_res = torch.optim.AdamW(res.parameters(), lr=1e-4, weight_decay=1e-4)
sch_res = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(opt_res, T_0=10)

for epoch in range(15):
    t0 = time.time()
    loss_tr = train_one_epoch(res, train_dl, opt_res, sch_res)
    val     = evaluate(res, val_dl)
    print(
        f"[{epoch:02d}] "
        f"loss_tr={loss_tr:.3f} | loss_val={val['loss']:.3f} | "
        f"Acc={val['acc']:.3%} | "
        f"F1(ma)={val['f1_macro']:.3%} | F1(we)={val['f1_weight']:.3%} | "
        f"Top-3={val['top3']:.3%} | time={time.time()-t0:.1f}s"
    )


### ViT-B/16

In [ ]:
from torchvision.models import vit_b_16, ViT_B_16_Weights
import time

vit = vit_b_16(weights=ViT_B_16_Weights.DEFAULT)
vit.heads.head = nn.Linear(vit.heads.head.in_features, NUM_CLASSES)
vit = vit.to(device)

opt_vit = torch.optim.AdamW(vit.parameters(), lr=5e-5, weight_decay=1e-4)
sch_vit = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(opt_vit, T_0=10)

for epoch in range(15):
    t0 = time.time()
    loss_tr = train_one_epoch(vit, train_dl, opt_vit, sch_vit)
    val     = evaluate(vit, val_dl)
    print(
        f"[{epoch:02d}] "
        f"loss_tr={loss_tr:.3f} | loss_val={val['loss']:.3f} | "
        f"Acc={val['acc']:.3%} | "
        f"F1(ma)={val['f1_macro']:.3%} | F1(we)={val['f1_weight']:.3%} | "
        f"Top-3={val['top3']:.3%} | time={time.time()-t0:.1f}s"
    )


In [ ]:
test_res_aug = evaluate(res, test_dl)
test_vit_aug = evaluate(vit, test_dl)


def show_cm(cm, title):
    import matplotlib.pyplot as plt
    plt.figure(figsize=(7, 6))
    plt.imshow(cm, interpolation="nearest")
    plt.title(title)
    plt.colorbar()
    ticks  = range(NUM_CLASSES)
    names  = sorted(train_df["class"].unique())
    plt.xticks(ticks, names, rotation=45, ha="right")
    plt.yticks(ticks, names)
    plt.ylabel("True"); plt.xlabel("Predicted")
    plt.tight_layout(); plt.show()


print("ResNet-18 + aug — eval_data:")
for k in ["acc", "f1_macro", "f1_weight", "top3"]:
    print(f"{k:9s}: {test_res_aug[k]:.3%}")
show_cm(test_res_aug["cm"], "ResNet-18 + aug — Confusion Matrix")

print("\nViT-B/16 + aug — eval_data:")
for k in ["acc", "f1_macro", "f1_weight", "top3"]:
    print(f"{k:9s}: {test_vit_aug[k]:.3%}")
show_cm(test_vit_aug["cm"], "ViT-B/16 + aug — Confusion Matrix")


#### Сравнивая с обыным бейзлайном, как будто ситуация осталась примерно такая же.